### Imports

In Deno, we have to prefix the imports from npmjs.org with `npm:`:

In [1]:
import { BufferMemory } from "npm:langchain/memory";
import { LLMChain } from "npm:langchain/chains";
import {
  PromptTemplate,
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "npm:@langchain/core/prompts";
import { StructuredOutputParser } from "npm:langchain/output_parsers";

import * as diff from 'npm:diff';
import { z } from "npm:zod";

import { html } from "https://deno.land/x/display/mod.ts";

## First step: we send our text to the LLM for review, and we get the model's response as a Javascript object containing the reviewed text

### Basic LangChain chains contain a prompt, a model and an output parser

#### Prompt

This is our prompt, it has two inputs: `instruction` and `inputText`. We'll assign values for these inputs, when we execute the chain

In [2]:
const prompt = PromptTemplate.fromTemplate(`
{instruction}
{inputText}
`);

### Chat model: codellama

We create an Ollama LLM model that uses the local [codellama:7b-code](https://ollama.com/library/codellama:7b-code) model. This is a small model, requires about 4 GB of RAM. I use this model on my older laptop that has only 16 GB RAM and a dual core CPU, and it gives responses with an acceptable speed. Bigger models, like [llama2:70b](https://ollama.com/library/llama2:70b) give better results, but they require a computer with 64 GB of RAM and a fast CPU/GPU.

In [3]:
import { Ollama } from "npm:@langchain/community/llms/ollama";
const llm = new Ollama({
  baseUrl: "http://localhost:11434",
  model: "codellama:7b-code",
  verbose: true,
});

### Chat model: OpenAI

With LangChain, we can use the same API with a wide variety of LLM models, so for example we can replace the `new ChatOllama(...)` part of the code with the following to use OpenAI's LLM model:

In [4]:
// import { OpenAI } from "npm:@langchain/openai";

// // auto-load .env
// import "https://deno.land/std@0.215.0/dotenv/load.ts";

// const llm = new OpenAI({ 
//   modelName: "gpt-3.5-turbo", 
//   verbose: true,
// });

### Output parser

We define a Zod schema. LangChain will parse the LLM model's output using this schema, and return a Javascript object

In [5]:
const outputParser = StructuredOutputParser.fromZodSchema(
  z.object({
    reviewedText: z.string()
  })
);

### We assembly the chain and execute it

In [19]:
// the chain contains the prompt, the model and the output parser
const chain = prompt.pipe(llm).pipe(outputParser);

// we specify the inputs of the prompt
const instruction = "Fix the grammar issues in the following text. Return the reviewed text in JSON only, using the following format: " +
    `{ "reviewedText": "the reviewed text" }`;
const inputText = "How to stays relevant as the developer in the world of ai?";

// finaly we execute the chain
const response = await chain.invoke({ instruction, inputText });

// and show the response from the LLM
console.log(response)

[llm/start] [1:llm:Ollama] Entering LLM run with input: {
  "prompts": [
    "Fix the grammar issues in the following text. Return the reviewed text in JSON only, using the following format: { \"reviewedText\": \"the reviewed text\" }\nHow to stays relevant as the developer in the world of ai?"
  ]
}
[llm/end] [1:llm:Ollama] [36.19s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "{\n\"reviewedText\":\"How to stay relevant as a developer in the world of AI?\"\n}"
      }
    ]
  ]
}
{
  reviewedText: "How to stay relevant as a developer in the world of AI?"
}


## Second step: we compare our original text and the LLM's response, and show the differences
#### We compare our original text and the LLM's response

In [20]:
const diffChanges = diff.diffChars(inputText, response.reviewedText);
console.log(diffChanges)

[
  { count: 11, value: "How to stay" },
  { count: 1, added: undefined, removed: true, value: "s" },
  { count: 13, value: " relevant as " },
  { count: 3, added: undefined, removed: true, value: "the" },
  { count: 1, added: true, removed: undefined, value: "a" },
  { count: 27, value: " developer in the world of " },
  { count: 2, added: undefined, removed: true, value: "ai" },
  { count: 2, added: true, removed: undefined, value: "AI" },
  { count: 1, value: "?" }
]


#### We show the differences between our original text and the LLM's response in HTML

In [21]:
const h = diffChanges.map((r) => {
  if (!r.added && !r.removed) {
    return `<span>${r.value}</span>`;
  } 
  if (r.added) {
    return `<span style="color: lightgreen;">${r.value}</span>`;
  } 
  if (r.removed) {
    return `<span style="color: red; text-decoration: line-through;">${r.value}</span>`;
  } 
}).join("");
    
html`<div style="background: black; color: white; padding: 10px;">${h}</div>`;

How to stay s relevant as the a developer in the world of ai AI ?